In [1]:
%%capture
from glob import glob
!pip install pytorch-lightning
import os
!pip install mne
import mne
import numpy as np
import pandas
import matplotlib.pyplot as plt

In [2]:
path=glob('dataverse_files-2/*.edf')
len(path)

28

In [3]:
healthyPath=[i for i in path if 'h' in i.split("/")[1]]
PatientPath=[i for i in path if 's' in i.split("/")[1]]

In [4]:
def read_data(filepath):
    data=mne.io.read_raw_edf(filepath,preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=1,h_freq=30)
    epoch=mne.make_fixed_length_epochs(data,duration=4,overlap=1)
    arr=epoch.get_data()
    return arr

In [5]:
%%capture
healthy_epochArr=[read_data(i) for i in healthyPath]
patient_epochArr=[read_data(i) for i in PatientPath]
control_epoch_labels=[len(i)*[0] for i in healthy_epochArr]
patient_epoch_labels=[len(i)*[1] for i in patient_epochArr]

In [6]:
data_list=healthy_epochArr+patient_epochArr
label_list=control_epoch_labels+patient_epoch_labels
group_list=[[i]*len(j) for i,j in enumerate (data_list)]

In [7]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import StandardScaler

class StandardScaler3D(BaseEstimator,TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self,X,y=None):
        self.scaler.fit(X.reshape(-1, X.shape[2]))
        return self

    def transform(self,X):
        return self.scaler.transform(X.reshape( -1,X.shape[2])).reshape(X.shape)

In [8]:
import numpy as np
data_array=np.concatenate(data_list)
label_array=np.concatenate(label_list)
group_array=np.concatenate(group_list)
data_array=np.moveaxis(data_array,1,2)

print(data_array.shape,label_array.shape,group_array.shape)

(9605, 1000, 19) (9605,) (9605,)


In [9]:
accuracy=[]
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler3D()
    train_features=scaler.fit_transform(train_features)
    val_features=scaler.transform(val_features)
    break

In [10]:
train_features.shape

(7662, 1000, 19)

In [11]:
from tensorflow.keras.layers import Input,Dense,concatenate,Flatten,GRU,Conv1D
from tensorflow.keras.models import Model

In [12]:
def block(input):
    conv1 = Conv1D(32, 2, strides=2,activation='relu',padding="same")(input)
    conv2 = Conv1D(32, 4, strides=2,activation='relu',padding="causal")(input)
    conv3 = Conv1D(32, 8, strides=2,activation='relu',padding="causal")(input)
    x = concatenate([conv1,conv2,conv3],axis=2)
    return x

In [13]:
input= Input(shape=(1000,19))
block1=block(input)
block2=block(block1)
block3=block(block2)

In [14]:
gru_out1 = GRU(32,activation='tanh',return_sequences=True)(block3)
gru_out2 = GRU(32,activation='tanh',return_sequences=True)(gru_out1)
gru_out = concatenate([gru_out1,gru_out2],axis=2)
gru_out3 = GRU(32,activation='tanh',return_sequences=True)(gru_out)
gru_out = concatenate([gru_out1,gru_out2,gru_out3])
gru_out4 = GRU(32,activation='tanh')(gru_out)


In [15]:
predictions = Dense(1,activation='softmax')(gru_out4)
model = Model(inputs=input, outputs=predictions)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [27]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
train_dataset = train_dataset.batch(128)  
val_dataset = tf.data.Dataset.from_tensor_slices((val_features, val_labels))
val_dataset = val_dataset.batch(128) 
model.fit(train_dataset, epochs=10,batch_size=128)

Epoch 1/10
60/60 [==============================] - 16s 267ms/step - loss: 0.7364 - accuracy: 0.5934
Epoch 2/10
60/60 [==============================] - 16s 265ms/step - loss: 0.7352 - accuracy: 0.5934
Epoch 3/10
60/60 [==============================] - 16s 265ms/step - loss: 0.7367 - accuracy: 0.5934
Epoch 4/10
60/60 [==============================] - 16s 265ms/step - loss: 0.7325 - accuracy: 0.5934
Epoch 5/10
60/60 [==============================] - 16s 264ms/step - loss: 0.7320 - accuracy: 0.5934
Epoch 6/10
60/60 [==============================] - 16s 272ms/step - loss: 0.7323 - accuracy: 0.5934
Epoch 7/10
60/60 [==============================] - 16s 266ms/step - loss: 0.7321 - accuracy: 0.5934
Epoch 8/10
60/60 [==============================] - 16s 273ms/step - loss: 0.7372 - accuracy: 0.5934
Epoch 9/10
60/60 [==============================] - 16s 272ms/step - loss: 0.7348 - accuracy: 0.5934
Epoch 10/10
60/60 [==============================] - 16s 270ms/step - loss: 0.7321 - accura

In [28]:
import tensorflow as tf
val_dataset = tf.data.Dataset.from_tensor_slices((val_features, val_labels))
batch_size = 128
val_dataset = val_dataset.batch(batch_size)
evaluation = model.evaluate(val_dataset)

16/16 [==============================] - 1s 77ms/step - loss: 0.8347 - accuracy: 0.3721


In [26]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1000, 19)]           0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 500, 32)              1248      ['input_1[0][0]']             
                                                                                                  
 conv1d_1 (Conv1D)           (None, 500, 32)              2464      ['input_1[0][0]']             
                                                                                                  
 conv1d_2 (Conv1D)           (None, 500, 32)              4896      ['input_1[0][0]']             
                                                                                              